<a href="https://colab.research.google.com/github/zachwhalen/nngm20/blob/main/GeneratingComics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generating Comics

This notebook contains code I used to generate my NaNoGenMo 2020 book, [_VAUD oc HORRRR_](https://github.com/nanogenmo/2020/issues/55). This is pretty messy. It worked for me, but I don't think it's necessarily easy for anyone else to do.

The big things that aren't here:
1. I used [multicrop2](http://www.fmwconcepts.com/imagemagick/multicrop2/index.php) to extract panels from comics. This required a lot of manual cleanup.
2. I used [Derrick Schultz's StyleGAN2 scripts](https://github.com/dvschultz) for create the `.pkl` files.

----

Start with some basic imports and set up.

In [ ]:
# imports
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import random
from glob import glob
import os.path
import cv2
import string
!pip install opensimplex
%tensorflow_version 1.x
%cd /content/drive/MyDrive/nngm20/
%mkdir 'work/panels'
%mkdir 'work/rows'
%mkdir 'work/rows/2'
%mkdir 'work/rows/3'

Assign the `.pkl` file to use and set some parameters that might need to be adjusted and/or reused later.

In [ ]:
gen = '/content/drive/MyDrive/nngm20/stylegan2-colab/stylegan2/run_generator.py'
pkl = '/content/drive/MyDrive/nngm20/stylegan2-colab/stylegan2/results/00020-stylegan2-ec-1gpu-config-f/network-snapshot-011946.pkl'
work_dir = './work'
seed_start = random.randint(0,50000000)
seed_end = seed_start + 8000

Use those settings to run the image generating script. This assumes you already have a well-trained `.pkl` file.


In [ ]:
!python {gen} generate-images --network={pkl} --seeds={seed_start}-{seed_end} --truncation-psi=0.5 --result-dir={work_dir}

Crop all of those generated images to remove the whitespace on the sides. Since the generated images are 1024x1024, the extra space needs to be removed so they can fit together.


In [ ]:

generated_images = glob("/content/drive/MyDrive/nngm20/work/00003-generate-images/*.png")

# crop all of the panels
for img in generated_images:

  fn = os.path.basename(img)
  # from here https://stackoverflow.com/a/59636960

  # Load image, grayscale, Gaussian blur, Otsu's threshold
  image = cv2.imread(img)
  original = image.copy()
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  blur = cv2.GaussianBlur(gray, (25,25), 0)
  thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]


  # Perform morph operations, first open to remove noise, then close to combine
  noise_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
  opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, noise_kernel, iterations=2)
  close_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
  close = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, close_kernel, iterations=3)

  # Find enclosing boundingbox and crop ROI
  coords = cv2.findNonZero(close)
  x,y,w,h = cv2.boundingRect(coords)
  cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 2)
  crop = original[y:y+h, x:x+w]

  ratio = crop.shape[1] / crop.shape[0]
  new_height = 1000
  new_width = int(ratio * 1000)

  # resize it to height = 1000 for standardized rows
  resized = cv2.resize(crop, (new_width,new_height), interpolation = cv2.INTER_AREA)

  done = cv2.imwrite('/content/drive/MyDrive/nngm20/work/panels/cropped-'+fn,resized)

Ingest all of the generated panels into a dictionary, `pd`, that contains their file name and width. This takes a long time. It could probably be done faster as a generator instead of a list, but this is fine I guess. With 8000 images, it took about an hour and a half.

In [ ]:
# run this to reset the pd database as needed
for p in pd.keys():
  pd[p]['used'] = 0

Arrange the panels into complete rows of appropriate width. `tw` sets that width to 2000, which seems to work. It starts with the largest-width panels and tries to fit them together until it runs out of possibilities.

In [ ]:
# target width
tw = 2000
fit = 100 # how close do the rows need to be to target width?
pad = 10 # how far apart should the panels be?
row_list = []
buckets = []
#keys = pd.keys()
def widthSort (x):
  return pd[x]['w']

sorted_keys = sorted(pd.keys(), key = widthSort, reverse = True)

for p in sorted_keys:
  if pd[p]['used'] is 0:
    for a in sorted_keys:
      if pd[a]['used'] is 0 and a is not p:
        nw = pd[p]['w'] + pd[a]['w']
        if (abs(nw - tw) < fit):
          row_list.append((nw,[p,a]))
          pd[p]['used'] = 1
          pd[a]['used'] = 1
          print(row_list[-1])
          break

print(len(row_list))

Group some of the remaining panels into three-panel rows. This took a few tries to find a method that would finish in a reasonable time. Basically it creates an arbitrary number of buckets, filling them in with random panels. Then it iterates through all of the unclaimed panels, sticking them in the first available bucket where the total width doesn't exceed 2000 +/- the `fit` variable.

In [ ]:
buckets = []
bucket_count = int(len(pd.keys())  - len(row_list) / 3)
ad = {x:pd[x] for x in list(pd.keys()) if pd[x]['used'] == 0}
a_sorted_keys = sorted(ad.keys(), key = widthSort, reverse = False)

for p in a_sorted_keys[:bucket_count]:
  buckets.append((pd[p]['w'],[p]))
  pd[p]['used'] == 1

bd = {x:pd[x] for x in list(pd.keys()) if pd[x]['used'] == 0}

b_sorted_keys = sorted(bd.keys(), key = widthSort, reverse = False)
for p in b_sorted_keys[:bucket_count]:
  for i in range(len(buckets)):
    bucket = buckets[i]
    if (bucket[0] + pd[p]['w'] < tw):
      bucket[1].append(p)
      buckets[i] = (bucket[0] + pd[p]['w'],bucket[1])
      pd[p]['used'] = 1
      break

cd = {x:pd[x] for x in list(pd.keys()) if pd[x]['used'] == 0}
c_sorted_keys = sorted(cd.keys(),key = widthSort, reverse = False)

for p in c_sorted_keys:
  for i in range(len(buckets)):
    bucket = buckets[i]
    if (bucket[0] + pd[p]['w'] < tw + fit):
      bucket[1].append(p)
      buckets[i] = (bucket[0] + pd[p]['w'],bucket[1])
      pd[p]['used'] = 1
      break

Choose the best three-panel rows by adjusting the threshold of the fit. I'm looking for a good balance between two- and three-panel rows.

In [ ]:
keepers = [x for x in buckets if x[0] > 1980]


Randomly arrange those panel rows into groups of three to compose pages.

In [ ]:
import random
chunking_list = row_list
pages = []

while (len(chunking_list) > 0):
  page = []
  for r in range(3):
    page.append(
        row_list.pop(random.randint(0,len(chunking_list) - 1))
    )
  pages.append(page)

Work through those page-groups, adjusting the row sizes so they're all the same. Add a page number, and stick that onto a static recto or verso page as needed.


In [ ]:
pad = 10
pn = 1

for page in pages:
  page_height = 0
  page_width = 1200
  row_images = []

  for row in page:
    #print(row)
    
    row_width = row[0] + (pad * len(row[1]))

    row_image = Image.new("RGBA",(row_width,1000),color=(0,0,0,0))

    h_offset = 0
    for p in row[1]:
      pim = Image.open(pd[p]['path'])
      progress = row_image.paste(pim,(h_offset,0))
      h_offset += pd[p]['w'] + pad

    row_image.thumbnail((1200,1200),resample = Image.BICUBIC)
    page_height += int(row_image.size[1] + (pad / 2))
    row_images.append(row_image)

  page_image = Image.new("RGBA",(1200,page_height),color=(0,0,0,0))

  offset = 0
  for r in row_images:
    #print(offset)
    progress = page_image.paste(r,(0,offset))
    offset += int(r.size[1] + (pad / 2))


  # add the page number
  template = '/content/drive/MyDrive/nngm20/work/page-number-template.png'
  font = ImageFont.truetype('/usr/share/fonts/truetype/liberation/LiberationSans-Bold.ttf', 11)

  base = Image.new("RGBA",(25,25),(0,0,0,0))
  draw = ImageDraw.Draw(base)
  npad = " " if (pn < 10) else ""
  draw.text((8,7), npad + str(pn),(50,50,50), font=font)
  temp = Image.open(template)
  temp1 = temp.convert("RGBA")
  base1 = base.resize((60,60),resample = Image.BICUBIC)
  done = Image.alpha_composite(temp1,base1)
  shim = Image.new("RGBA",(1200,page_height),color=(0,0,0,0))
  progress = shim.paste(done,(1200 - 63, page_height - 63))

  numbered_page = Image.alpha_composite(page_image,shim)

  recto = '/content/drive/MyDrive/nngm20/work/recto.png'
  verso = '/content/drive/MyDrive/nngm20/work/verso.png'

  shim = Image.new("RGBA",(1250,1920),color=(0,0,0,0))
  progress = shim.paste(numbered_page,(25,50))

  if (pn % 2 is 1):
    # odd, so add to recto
    base = Image.open(recto)
  else:
    # even, so add to verso
    base = Image.open(verso)

  merged_page = Image.alpha_composite(base,shim)

  file_name = str(pn)
  while (len(file_name) < 4):
    file_name = '0' + file_name

  progress = merged_page.save('/content/assemble/' + file_name + '.png')
  print(file_name + '.png')
  
  pn += 1


Generate a cover image.

In [ ]:
%mkdir cover
# generate a cover
!rm -rf '/content/cover/'


pkl = '/content/drive/MyDrive/nngm20/covers/stylegan2-colab/stylegan2/results/00002-stylegan2-covers-1gpu-config-f/network-snapshot-010163.pkl'
gen = '/content/drive/MyDrive/nngm20/stylegan2-colab/stylegan2/run_generator.py'
seed = random.randint(0,5000)

!python {gen} generate-images --network={pkl} --seeds={seed} --truncation-psi=0.3 --result-dir='/content/cover/'

[cover] = glob('/content/cover/00000-generate-images/*.png') 


im = Image.open(cover)
cropped = im.crop((170,10,1024-170,1024-10))

scaled = cropped.resize((1250,1835))

progress = scaled.save('/content/assemble/cover.png')


Finally, bind it all together into a PDF!

In [ ]:

pagelist = glob('/content/assemble/0*.png')
pagelist = sorted(pagelist)
pagelist = ['/content/drive/MyDrive/nngm20/work/verso.png'] + pagelist
page_images = []
for page in pagelist:
  im = Image.open(page).convert('RGB')
  page_images.append(im)

pdf = Image.open('/content/assemble/cover.png')

pdf.save('/content/book.pdf', "PDF" ,resolution=100.0, save_all=True, append_images=page_images)